# Synthetic Maritime Voyage Data Generation

This notebook generates synthetic voyage data capturing key uncertainty drivers:
- Weather impact (speed reduction)
- Port congestion delays
- Route deviation (distance increase)

Output:
- 'data/synthetic_voyages.csv'


In [ ]:
from src.simulation import simulate_eta_hours
from src.risk_metrics import summarize_risk
from src.plotting import plot_eta_distribution


: 

## Define voyage scenario

You can change the distance/speed/threshold to explore different risk profiles.


In [ ]:
distance_nm = 1200
expected_speed_knots = 14
late_threshold_hours = 110
n_sims = 20000


In [ ]:
samples = simulate_eta_hours(
    distance_nm=distance_nm,
    expected_speed_knots=expected_speed_knots,
    n_sims=n_sims,
    seed=7
)
len(samples), samples[:5]


In [ ]:
summary = summarize_risk(samples, late_threshold_hours=late_threshold_hours)
summary


In [ ]:
plot_eta_distribution(samples, save_path="reports/figures/eta_hist.png")
print("Saved figure: reports/figures/eta_hist.png")


## Interpretation

- **Mean**: expected ETA under uncertainty
- **P95**: conservative ETA for planning (only ~5% of runs exceed this)
- **Probability of late arrival**: risk of breaching an SLA threshold

These metrics help convert uncertainty into operational decisions.


# Stress Scenarios

We compare a normal day vs adverse conditions by changing uncertainty assumptions.


In [ ]:
# Normal
normal = simulate_eta_hours(distance_nm, expected_speed_knots, n_sims=20000, seed=1)

# Storm (worse weather uncertainty)
storm = simulate_eta_hours(
    distance_nm, expected_speed_knots, n_sims=20000, seed=2
)

# Port strike / high congestion (simulate by reducing effective speed a bit + more delay)
# Simple proxy: slightly lower speed to represent operational disruption
port_strike = simulate_eta_hours(
    distance_nm, expected_speed_knots * 0.92, n_sims=20000, seed=3
)

summaries = {
    "normal": summarize_risk(normal, late_threshold_hours),
    "storm": summarize_risk(storm, late_threshold_hours),
    "port_strike": summarize_risk(port_strike, late_threshold_hours),
}
summaries


In [ ]:
import pandas as pd
pd.DataFrame(summaries).T


### Scenario takeaway

Compare **P95** and **probability of late arrival** across scenarios to understand operational risk under stress.
